<a href="https://colab.research.google.com/github/TottiPuc/Machine_learning/blob/master/An%C3%A1lisis_sentimiento_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Análisis de sentimientos en reseñas de peliculas usando Pytorch
En este notebook usaremos pytorch para realizar tareas de PLN y crear una red neuronal que pueda extraer las caracteristicas de un texto y predecir si es un comentario bueno o malo sobre una pelicula.
Usaremos el dataset publico de imdb disponible en [dataset_peliculas](https://www.imdb.com/interfaces/) ademas de las etiquetas o labels de los sentimientos que pueron extraidos del portal [sentimientos](https://ai.standford.edu/~amaas/data/sentiment/) 

In [0]:
!pip install watermark

In [0]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import sklearn
from torch.utils.data import DataLoader, Dataset
from sklearn.feature_extraction.text import CountVectorizer
from tqdm.notebook import tqdm, tqdm_notebook

In [4]:
%reload_ext watermark
%watermark -a "Christian Dayan Arcos Gordillo" --iversion

pandas  1.0.3
sklearn 0.22.2.post1
torch   1.4.0
Christian Dayan Arcos Gordillo


In [0]:
#definiendo el trabajo con GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
device

device(type='cuda')

##Cargando datasets

In [7]:
from google.colab import files
files.upload()

Saving imdb_reviews.csv to imdb_reviews.csv


{'imdb_reviews.csv': b'A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  \t0\nNot sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  \t0\nAttempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  \t0\nVery little music or anything to speak of.  \t0\nThe best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.  \t1\nThe rest of the movie lacks art, charm, meaning... If it\'s about emptiness, it works I guess because it\'s empty.  \t0\nWasted two hours.  \t0\nSaw the movie today and thought it was a good effort, good messages for kids.  \t1\nA bit predictable.  \t0\nLoved the casting of Jimmy Buffet as the science teacher.  \t1\nAnd those baby owls were adorable.  \t1\nThe movie showed a lot of Florida at it\'s best, made it look very 

In [0]:
names = ['Review','Sentiments']
data = pd.read_csv('imdb_reviews.csv', sep='\t', names= names)

In [9]:
data.head()

,Review,Sentiments
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [10]:
data.shape

(748, 2)

In [12]:
#verificando las porciones de sentimientos para ver si las clases estan bien balanceadas
data['Sentiments'].value_counts()

1    386
0    362
Name: Sentiments, dtype: int64

Manipulacion de texto usando vectorizacion esto es convertir una coleccion de texto en una matriz de tokens lo que genera una matriz esparsa 

In [0]:
#Creamos el vectorizador
class Sequences():
  def __init__(self):
    self.vectorizer = CountVectorizer(stop_words= 'english', max_df=0.99, min_df=0.005)
    self.sequences = self.vectorizer.fit_transform(data.Review.tolist())
    self.labels = data.Sentiments.tolist()  
    self.token2idx = self.vectorizer.vocabulary_ 
    self.idx2token = {idx: token for token, idx in self.token2idx.items()}    

  def __getitem__(self,i):
    return self.sequences[i,:].toarray(),self.labels[i]

  def __len__(self):
    return self.sequences.shape[0]                  

instanciamos el modelo para extraer las frases del dataset y crear la matriz de datos con los tokens, conteo de cada token y etiquetas

In [0]:
data_sentences = Sequences()

In [30]:
print(data_sentences[5][0].shape)

(1, 320)


Preparando los daots para entrenamiento en el formato Pytorch

In [31]:
train_loader = DataLoader(data_sentences, batch_size=4096)
train_loader

##Definición e contrucción del modelo

In [0]:
#clasificador
class BagOfWordsClassifier(nn.Module):
  #metodo constructor para inicializar los atributos
  def __init__(self)